# Directive for creating a script for your notebook

The block here below is required at the top of each notebook that you want to create a script for. You will also need to edit the "settings.ini" file, to create a script (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details). Replace **some_string** with a name that makes sense for your notebook. 

In [ ]:
# |default_exp Nmeningitidis_parser


# Libraries
Include all the libraries which should be used in this module. You can also import modules from other notebooks; here, we have imported the functions in the core notebook.

In [ ]:
# |export

# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script
import json  # for nicely printing json and yaml

#import functions from core module (optional, but most likely needed). 
from ssi_analysis_result_parsers import(
    core,
    blast_parser,
)
#from ssi_analysis_result_parsers.blast_parser import extract_presence_absence

# Project specific libraries
from pathlib import Path
import pandas
import numpy
import sys

In [ ]:
# This block should never be exported. It is to have python running in the project (and not the nbs) dir, and to initiate the package using pip.
os.chdir(core.PROJECT_DIR)

# Functions

Add your code here below. If your notebook will be used as a console-script, you need to add a "cli"-function, at the end (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Code execution** and **Input, output and options**) on loop for more details)

In [ ]:
# | export


def extract_meningotype(meningotype_tsv: Path):
    if meningotype_tsv.exists():
        try:
            df = pandas.read_csv(meningotype_tsv, sep='\t')
            PorA_split = df["PorA"][0].split(",")
            return {"SEROGROUP": df["SEROGROUP"][0],
                    "VR1": PorA_split[0],
                    "VR2": PorA_split[1],
                    "FetA": df["FetA"][0],
                    }
        except pandas.errors.EmptyDataError:
            print(f"Meningotype output file empty at {meningotype_tsv}", file=sys.stderr)
            return None
    else:
        print(f"No meningotype output found at {meningotype_tsv}", file=sys.stderr)
        return None

    return None

def extract_cc_from_mlst(MLST: str, mlst_scheme_tsv: Path = "/users/data/Projects/MICROBIAL_SURVEILLANCE/Resources/Databases/mlst/neisseria/neisseria.txt"):
    CC = {"CC": ""}
    if mlst_scheme_tsv.exists():
        try:
            with open(mlst_scheme_tsv) as f:
                for line in f:
                    line = line.strip('\n').split('\t')
                    if line[0] == MLST:
                        if line[8][:3] == "ST-":
                            CC["CC"] = line[8].split(" ")[0][3:]
                        return CC
        except Exception as e:
            CC["CC"] = f"Failed to load mlst scheme tsv: {e}"
    else:
        CC["CC"] = "Failed to load mlst scheme tsv, file not found"
    return CC


class NmeningitidisResults(core.PipelineResults):

    @classmethod
    def from_tool_paths(cls, meningotype_tsv: Path, MLST: str, mlst_scheme_tsv: Path, sample_name = None):
        """
        Alternative constructor for initializing results for single sample,
        Initializes NmeningitidisResults instance provided paths to outputs from tools (legionella sbt and lag1 presence blast)
        """
        mgk_results = cls.summary(meningotype_tsv=Path(meningotype_tsv),
                                  MLST=MLST,
                                  mlst_scheme_tsv=Path(mlst_scheme_tsv))
        return cls( {sample_name: mgk_results})
    
    @classmethod
    def from_tool_paths_dict(cls, file_paths: dict):
        """
        Alternative constructor for initializing results for multiple samples,
        Initializes NmeningitidisResults instance by providing a dictionary of paths to outputs from tools (legionella sbt and lag1 presence blast)
        """
        results_dict = {}
        for sample_name, path_dict in file_paths.items():
            mgk_results = cls.summary(meningotype_tsv=Path(path_dict["meningotype_results"]),
                                      MLST=path_dict["MLST"],
                                      mlst_scheme_tsv=path_dict["mlst_scheme_tsv"]
                                      )
            results_dict[sample_name] = mgk_results
        return cls(results_dict)
    
    @classmethod
    def from_tool_paths_dataframe(cls, file_paths_df: pandas.DataFrame):
        """
        Alternative constructor for initializing results for multiple samples,
        Initializes NmeningitidisResults instance by providing a DataFrame of paths to outputs from tools (legionella sbt and lag1 presence blast)
        """
        file_paths = file_paths_df.to_dict(orient="index")
        
        return cls.from_tool_paths_dict(file_paths=file_paths)

    @classmethod
    def from_tool_paths_tsv(cls, tool_paths_tsv: Path):
        """
        Alternative constructor for initializing results for multiple samples,
        Initializes NmeningitidisResults instance by providing a tsv-file with paths to outputs from tools (legionella sbt and lag1 presence blast)
        """
        file_paths_df = pandas.read_csv(tool_paths_tsv, sep='\t')
        file_paths_df.set_index("sample_name", inplace=True, drop=True)
        return cls.from_tool_paths_dataframe(file_paths_df)
    
    @staticmethod
    def summary(meningotype_tsv: Path, MLST: str, mlst_scheme_tsv: Path) -> dict:
        meningotype_results = extract_meningotype(meningotype_tsv=Path(meningotype_tsv))
        results_dict = meningotype_results
        cc_dict = extract_cc_from_mlst(MLST=MLST, mlst_scheme_tsv=Path(mlst_scheme_tsv))
        results_dict = core.update_results_dict(results_dict, cc_dict, old_duplicate_key_prefix="Clonal_complex: ")
        if results_dict is None:
            return {}
        return results_dict


    def __repr__(self):
        return(f"< Spyogenes analysis results object. {len(self.results_df)} samples with {len(self.results_df.columns)} result variables > ")




## TESTING


In [ ]:

meningotype_results = extract_meningotype(meningotype_tsv=Path("test_input/Nmeningitidis/meningotype/meningotype1.tsv"))

assert(meningotype_results["SEROGROUP"] == "Y")
assert(meningotype_results["VR1"] == "21")
assert(meningotype_results["VR2"] == "16")
assert(meningotype_results["FetA"] == "F3-7")

cc_results = extract_cc_from_mlst("1157",Path("test_input/Nmeningitidis/neisseria_mlst_scheme.tsv"))
cc_results = extract_cc_from_mlst("11",Path("test_input/Nmeningitidis/neisseria_mlst_scheme.tsv"))




f = NmeningitidisResults.from_tool_paths_dict(file_paths=  {"sample_1": {"meningotype_results": "test_input/Nmeningitidis/meningotype/meningotype1.tsv", "MLST": "1466", "mlst_scheme_tsv": "test_input/Nmeningitidis/neisseria_mlst_scheme.tsv"},
                                                            "sample_2": {"meningotype_results": "test_input/Nmeningitidis/meningotype/meningotype2.tsv", "MLST": "1157", "mlst_scheme_tsv": "test_input/Nmeningitidis/neisseria_mlst_scheme.tsv"}
                                                            })

assert(f.results_df["SEROGROUP"]["sample_1"] == "Y")
assert(f.results_df["VR1"]["sample_1"] == "21")
assert(f.results_df["CC"]["sample_1"] == "174")
assert(f.results_df["SEROGROUP"]["sample_2"] == "X")
assert(f.results_df["VR2"]["sample_2"] == "2-67")
assert(f.results_df["CC"]["sample_2"] == "1157")

f = NmeningitidisResults.from_tool_paths_tsv(tool_paths_tsv=Path("test_input/Nmeningitidis/batch_parser_file_paths.tsv"))

print(f.results_df)

         SEROGROUP   VR1   VR2   FetA    CC
sample_1         Y    21    16   F3-7   174
sample_2         X     5  2-67  F5-36  1157
sample_3         B  18-1  30-4   F3-3      
sample_4       NaN   NaN   NaN    NaN      
sample_5       NaN   NaN   NaN    NaN      


No meningotype output found at test_input/Nmeningitidis/meningotype/meningotype4.tsv
Meningotype output file empty at test_input/empty_file.txt


In [ ]:
# |export



@call_parse
def Nmeningitidis_parser(
    meningotype_tsv: Path = None,  # Blast output from blasting EMM and emm-like genes
    MLST: str = None,  # MLST to deduce Clonal complex from
    mlst_scheme_tsv: Path = None, # Path to pubmlst scheme for neisseria for MLST: CC table
    sample_name: str = None,
    output_file: Path = None
) -> None:
    """
    
    """
    results = NmeningitidisResults.from_tool_paths(meningotype_tsv=meningotype_tsv,
                                                   MLST=MLST,
                                                   mlst_scheme_tsv=mlst_scheme_tsv,
                                                   sample_name=sample_name)
    results.write_tsv(output_file=output_file)

@call_parse
def Nmeningitidis_batch_parser(
    file_path_tsv: Path = None,  # Path to tsv containing file paths to the outputs from tools to be parsed. Must contain headers "sample_name", "sbt_results", and "lag1_blast_results"
    output_file: Path = None,  # Path to output tsv
) -> None:
    """
    
    """
    results = NmeningitidisResults.from_tool_paths_tsv(tool_paths_tsv=file_path_tsv)
    results.write_tsv(output_file)



# Directive for ensuring that the code in your notebook get executed as a script

The code-block here below is required to ensure that the code in the notebook is also transferred to the module (script), otherwise it will just be a notebook. See [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details.

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()